In [12]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle
import io
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import median_test


# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from find_significant import find_significant

In [17]:
stat,p,m,table = median_test(Data.sscl,Data.ssdl,ties='above',nan_policy='omit')
stat,p,m,table

(0.14689808042692354,
 0.701517464865618,
 0.9927459999999999,
 array([[1235, 1247],
        [1249, 1232]], dtype=int64))

# **Loading Data**

In [45]:
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM")
Data = pd.read_csv('Data.csv')
cond = np.logical_and(Data.year>=1390, Data.year<=1397)
Data = Data[cond]

In [46]:
OCM = ['sum_over1','sumfive','sumfour','sumthree','sumtwo','largest_owner','first_second',
       'first_sumtwofour', 'gini','herfindahl','sscl', 'ssco', 'ssdl', 'ssdo', 'bzcl', 'bzco', 'bzdl']

Vars = ['marketcap','payout', 'DIVTOI', 'DIVTNI', 'DIVTEQ', 'DIVTS', 'DPAY',
        'DIVTMVE', 'ROA', 'ROE', 'DLOSS', 'NPLOSS', 'MTB', 'LEVRG', 'LNTA','LNQ']

describe_columns = [('above_meadian','')]+list(zip(Vars,['mean']*len(Vars)))

# **Univariate Analysis**

In [33]:
%%time
iterables = [Vars, ['High', 'Low','diff','p_value']]
col_index = pd.MultiIndex.from_product(iterables)
out = pd.DataFrame(index=OCM,columns=col_index)

for ocm in OCM:     
        
    df = Data.copy()
    
    meadian = df[ocm].median()
    df['above_meadian'] = df[ocm]>meadian
    df = df[~pd.isnull(df.above_meadian)]
    
    
    for var in Vars:
        
        temp = df[~pd.isnull(df[var])]
        serie1 = temp[temp.above_meadian][var]
        high = np.round(serie1.median(),3)
        
        serie2 = temp[~temp.above_meadian][var]
        low = np.round(serie2.median(),3)
        try:
            stat,p,m,table = median_test(serie1,serie2,ties='above',nan_policy='omit')
        except:
            stat,p,m,table = np.nan, np.nan, np.nan, np.nan
        
        diff = find_significant(p,np.round(high-low,3))
        
        out.loc[ocm,(var,)] = [high,low,diff,np.round(p,2)]
        
out         

C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2362: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = fn(*args, **kwargs)


Wall time: 5.64 s


marketcap                                      payout  \
                         High          Low            diff p_value   High   
sum_over1         2.32689e+06  1.45413e+06     872763.0***       0  0.762   
sumfive           2.26864e+06  1.51348e+06     755160.0***       0  0.749   
sumfour            2.1728e+06  1.57359e+06   599210.973***       0  0.733   
sumthree          2.01963e+06  1.71778e+06    301854.724**    0.04  0.722   
sumtwo             1.8846e+06  1.94861e+06        -64008.0    0.76  0.699   
largest_owner     1.65668e+06  2.26864e+06    -611965.0***       0  0.683   
first_second      1.52076e+06  2.48493e+06    -964172.5***       0  0.603   
first_sumtwofour  1.45413e+06  2.61526e+06   -1161133.0***       0  0.595   
gini               1.6947e+06  2.21791e+06  -523211.006***       0  0.653   
herfindahl         1.7589e+06   2.1148e+06      -355900.0*    0.07  0.701   
sscl              1.65668e+06  2.26864e+06    -611965.0***       0  0.683   
ssco              2.22818e+06  1.69032e+06     537855.0***       0  0.541   
ssdl               1.6494e+06  2.27751e+06    -628113.0***       0  0.685   
ssdo              2.22884e+06  1.69224e+06     536597.5***       0   0.54   
bzcl                      NaN  1.89907e+06             nan     NaN    NaN   
bzco               2.4265e+06  1.63752e+06     788975.0***       0  0.577   
bzdl                      NaN  1.89907e+06             nan     NaN    NaN   

                                           DIVTOI         ...      LEVRG  \
                    Low       diff p_value   High    Low  ...       diff   
sum_over1          0.44   0.322***       0  0.568  0.333  ...    0.027**   
sumfive           0.462   0.287***       0  0.548  0.366  ...   0.042***   
sumfour           0.475   0.258***       0  0.532   0.38  ...   0.045***   
sumthree          0.491   0.231***       0  0.519  0.404  ...   0.052***   
sumtwo            0.516   0.183***       0    0.5  0.443  ...   0.072***   
largest_owner     0.556   0.127***       0  0.485  0.454  ...   0.066***   
first_second       0.65     -0.047    0.23  0.451  0.504  ...   0.028***   
first_sumtwofour  0.655     -0.06*    0.06  0.445  0.514  ...   0.032***   
gini              0.596    0.057**    0.02  0.477   0.47  ...    0.026**   
herfindahl        0.529   0.172***       0  0.501  0.445  ...   0.075***   
sscl              0.556   0.127***       0  0.485  0.454  ...   0.066***   
ssco              0.692  -0.151***       0  0.446  0.494  ...  -0.062***   
ssdl              0.556   0.129***       0  0.486  0.454  ...   0.065***   
ssdo               0.69   -0.15***       0  0.446  0.494  ...  -0.063***   
bzcl              0.625        nan     NaN    NaN  0.475  ...        nan   
bzco              0.662  -0.085***       0  0.465  0.478  ...  -0.065***   
bzdl              0.625        nan     NaN    NaN  0.475  ...        nan   

                            LNTA                               LNQ         \
                 p_value    High     Low       diff p_value   High    Low   
sum_over1           0.01  14.526  13.841   0.685***       0  1.112  0.978   
sumfive                0  14.498   13.87   0.628***       0  1.122  0.949   
sumfour                0  14.476  13.902   0.574***       0  1.117  0.973   
sumthree               0  14.424  13.983   0.441***       0  1.116  0.975   
sumtwo                 0   14.38  14.054   0.326***       0  1.124  0.973   
largest_owner          0  14.222  14.185      0.037    0.78   1.11   0.99   
first_second        0.01  14.104  14.333  -0.229***    0.01   1.02  1.095   
first_sumtwofour       0  14.056  14.425  -0.369***       0   1.02  1.098   
gini                0.02   14.19  14.218     -0.028    0.78  1.029  1.088   
herfindahl             0  14.293  14.099    0.194**    0.02  1.102  0.997   
sscl                   0  14.222  14.185      0.037    0.78   1.11   0.99   
ssco                   0  14.164  14.243     -0.079    0.46  0.972  1.116   
ssdl                   0   14.22 

In [35]:
out[('DIVTOI',)]

C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


,High,Low,diff,p_value
sum_over1,0.568,0.333,0.235***,0
sumfive,0.548,0.366,0.182***,0
sumfour,0.532,0.38,0.152***,0
sumthree,0.519,0.404,0.115***,0
sumtwo,0.5,0.443,0.057**,0.01
largest_owner,0.485,0.454,0.031,0.23
first_second,0.451,0.504,-0.053**,0.04
first_sumtwofour,0.445,0.514,-0.069***,0
gini,0.477,0.47,0.007,0.8
herfindahl,0.501,0.445,0.056**,0.01


## Latex Output

In [36]:
index = ['Sum_over1','Sumfive','Sumfour','Sumthree','Sumtwo','Largest Owner','First/Second',
         'First/Sumtwofour', 'Gini','Herfindahl','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']

out['OCM'] = index
out.set_index('OCM',inplace=True)

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Vars and OCM Meadian\PythonTex")
for var in Vars:
    temp = out[(var,)]
    
    file_name = var+'.tex'
    caption = var+' for Two Groups of Above and Below Meadian'
    
    temp.to_latex(file_name,index=True,multirow=True,
             bold_rows=True,caption=caption)
    

C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


# **Regression Analysis**

In [37]:
# These Two file are creatied using the do-file "Regressions and OCM-1.do" in
# "C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Vars and OCM Meadian"

col_names = ["DIVTOI", "DIVTNI", "DIVTS"]
row_index = ["largest_owner", "first_second", "first_sumtwofour", "sumfive", "herfindahl",
             "gini", "sscl", "ssco", "ssdl", "ssdo", "bzcl", "bzco", "bzdl"]

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Vars and OCM Meadian")
Coef = pd.read_excel('Coefs.xlsx',header=None,sheet_name='Coef',names=col_names)
Coef['row_index'] = row_index
Coef = Coef.set_index('row_index')

p_val = pd.read_excel('Coefs.xlsx',header=None,sheet_name='p_val',names=col_names)
p_val['row_index'] = row_index
p_val = p_val.set_index('row_index')

In [38]:
out = pd.DataFrame(index=row_index,columns=col_names)
for row in row_index:
    for col in col_names:
        out.loc[row,col] = find_significant(p=p_val.loc[row,col],val=Coef.loc[row,col])
        
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Vars and OCM Meadian\PythonTex")
out.to_latex('Regression and Vars.tex',index=True,multirow=True,
             bold_rows=True,caption="Coefficients of the OCM in the Regression")        

# **Yearly Univariate Analysis**

In [47]:
%%time
iterables = [Vars,list(Data.year.drop_duplicates()), ['High', 'Low','diff','p_value']]
col_index = pd.MultiIndex.from_product(iterables)
out_yearly = pd.DataFrame(index=OCM,columns=col_index)

iterables = [Vars,list(Data.year.drop_duplicates())]
col_index = pd.MultiIndex.from_product(iterables)
out_yearly_diff = pd.DataFrame(index=OCM,columns=col_index)


for ocm in OCM:     
        
    df = Data.copy()
    
    for year in list(Data.year.drop_duplicates()):
        df_year = df[df.year==year]
        meadian = df_year[ocm].median()
        df_year['above_meadian'] = df_year[ocm]>meadian
        df_year = df_year[~pd.isnull(df_year.above_meadian)]
        
        for var in Vars:
        
            temp = df_year[~pd.isnull(df_year[var])]
            
            serie1 = temp[temp.above_meadian][var]
            high = np.round(serie1.median(),3)

            serie2 = temp[~temp.above_meadian][var]
            low = np.round(serie2.median(),3)

            try:
                stat,p,m,table = median_test(serie1,serie2,ties='above',nan_policy='omit')
            except:
                stat,p,m,table = np.nan, np.nan, np.nan, np.nan

            diff = find_significant(p,np.round(high-low,3))

            out_yearly.loc[ocm,(var,year,)] = [high,low,diff,np.round(p,2)]
            out_yearly_diff.loc[ocm,(var,year)] = diff
            
            
out_yearly      
out_yearly_diff

C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2362: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = fn(*args, **kwargs)


Wall time: 39.1 s


marketcap                                               \
                        1390           1391           1392           1393   
sum_over1           310212.0    647865.632*       712442.0      505192.5*   
sumfive            316200.0*     557434.456       529760.0       321739.0   
sumfour           211687.785     560366.756       455820.0       136280.4   
sumthree           217811.22    632645.632*       85166.75       185316.4   
sumtwo               21866.0       36786.72    -149389.265       2077.344   
largest_owner      -264744.0     -677560.0*    -786480.0**    -317989.15*   
first_second      -327112.0*  -1231848.0***    -740190.0**  -616409.325**   
first_sumtwofour  -432110.0*  -1290472.0***  -1093320.0***   -889299.0***   
gini                -91850.0    -613131.562    -258364.776   -486113.638*   
herfindahl        -121576.25    -470412.625    -228253.265      -99760.56   
sscl               -264744.0    -872913.0**            nan            nan   
ssco              237418.514       594440.0     614200.0**      240409.99   
ssdl               -272196.3      -647120.0            nan            nan   
ssdo              247581.257       612070.0       522690.0     212616.681   
bzcl              -322940.0*            nan            nan            nan   
bzco                299288.6   1238238.0***    849840.0***   486113.638**   
bzdl                     nan            nan            nan            nan   

                                                                 \
                           1394           1395             1396   
sum_over1           681900.0***    1040672.0**       950883.368   
sumfive              577100.0**     1019187.0*         826450.0   
sumfour               518220.0*     1006325.0*        627986.52   
sumthree               316519.0       775069.3         614280.0   
sumtwo                 82984.88       533618.4       -377373.42   
largest_owner      -678058.15**     -553414.0*      -651869.622   
first_second        -897660.0**   -922438.0***   -1249050.18***   
first_sumtwofour  -1052850.0***  -1112242.0***  -1316996.678***   
gini                -384214.191     -485180.0*      -899195.44*   
herfindahl            -187730.0    -143091.047        -504325.0   
sscl                        nan            nan              nan   
ssco               650782.498**      506900.0*       622529.804   
ssdl                        nan            nan              nan   
ssdo                  586350.0*      524254.0*         490250.0   
bzcl                        nan            nan              nan   
bzco                670275.646*   644663.511**       981448.66*   
bzdl                        nan            nan              nan   

                                     payout            ...      LNTA  \
                             1397      1390      1391  ...      1396   
sum_over1         13416825.241***   0.198**  0.271***  ...   0.71***   
sumfive           11823679.543***   0.198**   0.212**  ...  0.783***   
sumfour           11823679.543***   0.225**   0.197**  ...  0.731***   
sumthree              7083061.972   0.208**   0.191**  ...  0.608***   
sumtwo                  2630531.0   0.155**     0.094  ...   0.261**   
largest_owner          -1082762.5    0.142*     0.076  ...     0.193   
first_second         -6563775.0**      0.02     0.002  ...    -0.304   
first_sumtwofour     -5908732.5**     0.012     -0.08  ...    -0.247   
gini                  -4667040.0*   0.143**     0.025  ...    -0.054   
herfindahl             624273.475    0.2***   0.126**  ...   0.279**   
sscl                          nan   0.149**     0.076  ...       nan   
ssco                    -228300.0  -0.162**  -0.172**  ...    -0.238   
ssdl                          nan     0.13*     0.099  ...       nan   
ssdo                  -1020246.95  -0.167**  -0.157**  ...    -0.238   
bzcl                          nan     0.106       nan  ...       nan   
bzco                  1725419.833   -0.116*     -0.04  ..

In [50]:
out_yearly[('ROA',1397)]
out_yearly_diff[('payout',)]

,High,Low,diff,p_value
sum_over1,0.139,0.069,0.07***,0
sumfive,0.145,0.066,0.079***,0
sumfour,0.143,0.069,0.074***,0
sumthree,0.134,0.072,0.062***,0
sumtwo,0.134,0.081,0.053**,0.01
largest_owner,0.126,0.104,0.022,0.13
first_second,0.106,0.133,-0.027,0.11
first_sumtwofour,0.11,0.12,-0.01,0.44
gini,0.114,0.115,-0.001,0.96
herfindahl,0.126,0.093,0.033,0.11


C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


,1390,1391,1392,1393,1394,1395,1396,1397
OCM,,,,,,,,
Sum_over1,0.198**,0.271***,0.224***,0.147**,0.418***,0.497***,0.569***,0.481***
Sumfive,0.198**,0.212**,0.221***,0.152*,0.418***,0.489***,0.515***,0.472***
Sumfour,0.225**,0.197**,0.213***,0.18*,0.418***,0.44***,0.477***,0.457***
Sumthree,0.208**,0.191**,0.216***,0.172**,0.361***,0.428***,0.405***,0.418***
Sumtwo,0.155**,0.094,0.11*,0.132,0.243**,0.284***,0.311***,0.358***
Largest Owner,0.142*,0.076,0.107,0.078,0.092,0.117,0.258**,0.24*
First/Second,0.02,0.002,-0.039,-0.037,-0.038,-0.083,0.068,-0.077
First/Sumtwofour,0.012,-0.08,-0.031,-0.037,-0.067,-0.074,0.075,-0.051
Gini,0.143**,0.025,0.097,0.064,0.167,0.061,0.197,0.015


## Latex Output

In [51]:
index = ['Sum_over1','Sumfive','Sumfour','Sumthree','Sumtwo','Largest Owner','First/Second',
         'First/Sumtwofour', 'Gini','Herfindahl','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']

out_yearly_diff['OCM'] = index
out_yearly_diff.set_index('OCM',inplace=True)

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Vars, OCM, Summary Through Time\PythonTex")
for var in Vars:
    temp = out_yearly_diff[(var,)]
    
    file_name = var+'_Yearly.tex'
    caption = 'Differences in median of '+var+' for Two Groups of Above and Below Meadian'
    
    temp.to_latex(file_name,index=True,multirow=True,
             bold_rows=True,caption=caption)
    

C:\Users\Mahdi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
